## Resumen del problema
Entrenar dos modelos de IA, A y B, maximizando el impacto, con las siguientes condiciones:

* Modelo A genera 5 puntos de impacto por hora.
* Modelo B genera 3 puntos de impacto por hora.
* Se dispone de 20 horas máximas en total.
* El modelo A no puede usarse más de 8 horas.

### Formulación matemática del problema
Función objetivo a maximizar:

$$
\quad Z = 5x + 3y
$$

donde:

* $𝑥$: horas dedicadas al Modelo A
* $y$: horas dedicadas al Modelo B

### Sujeto a las restricciones:
$$
\begin{aligned}
\quad &
\left\{
\begin{array}{l}
x + y \leq 20 \quad &\text{(máximo total de horas)} \\
x \leq 8 \quad &\text{(límite para el modelo A)} \\
x \geq 0,\ y \geq 0 \quad &\text{(no se pueden dedicar horas negativas)}
\end{array}
\right.
\end{aligned}
$$

### Importar librerías

In [ ]:
# librerías para cálculos numéricos y para graficar
import numpy as np
import matplotlib.pyplot as plt

# La función linprog de la librería scipy.optimize permite resolver problemas de programación lineal, es decir,
# encontrar el valor óptimo (máximo o mínimo) de una función lineal sujeta a restricciones también lineales.
from scipy.optimize import linprog

### Coeficientes de la función objetivo (negativos porque linprog minimiza)

In [ ]:
# Se quiere maximizar: Z = 5x + 3y
# Como linprog solo minimiza, convertimos el problema en: minimizar -Z = -5x -3y
c = [-5, -3]

# Coeficientes de las restricciones en forma matricial Ax <= b
# Restricciones:
# x + y <= 20  (máximo total de horas)
# x <= 8       (modelo A no se puede usar más de 8 horas)
A = [
    [1, 1],  # Coeficientes de la primera restricción: x + y <= 20
    [1, 0],  # Coeficientes de la segunda restricción: x <= 8
]

# Lado derecho de las restricciones
b = [20, 8] # Valores correspondientes a las desigualdades anteriores

# Límites para las variables (ambas deben ser >= 0)
x_bounds = (0, None)  # x ≥ 0 (sin límite superior)
y_bounds = (0, None)  # y ≥ 0

### Resolver el problema

In [ ]:
# Se ejecuta el solver de programación lineal
# - c: función objetivo (ya negativa para maximizar)
# - A_ub y b_ub: restricciones en forma Ax <= b
# - bounds: límites inferiores y superiores para cada variable
# - method: 'highs' es el método moderno y eficiente por defecto en SciPy
# - A_ub (Upper Bound) significa que está limitado por arriba, por una cota superior.
res = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds], method='highs')

### Mostrar resultados

In [ ]:
# Verifica si se encontró una solución óptima
if res.success:
    # Extraer los valores óptimos de las variables
    x, y = res.x

    # Mostrar resultados
    print(f"Horas para Modelo A: {x:.2f}")
    print(f"Horas para Modelo B: {y:.2f}")
    print(f"Impacto total máximo: {5*x + 3*y:.2f}")
else:
    # Si no se pudo encontrar solución (por restricciones inconsistentes, por ejemplo)
    print("No se encontró solución óptima.")

### ¿Por qué esto es IA?
Este ejemplo se relaciona con IA porque:

* Optimiza recursos computacionales (tiempo de entrenamiento).
* Ayuda a tomar decisiones automáticas bajo restricciones (planificación).
* Ilustra cómo aplicar técnicas de programación matemática para apoyar tareas en aprendizaje automático.

### Representación gráfica

Restricciones:
* $x + y <= 20  →  y <= 20 - x$
* $x <= 8$  → es una línea vertical en $x = 8$
* Restricción implícita: $y >= 0$ y $x >= 0 →$ sólo parte positiva


In [ ]:
# Crea un conjunto de 200 puntos entre 0 y 20. Se usa para graficar líneas continuas en el eje X
x = np.linspace(0, 20, 200)

# Restricciones
y1 = 20 - x

### Dibujar el área factible sombreada

Las listas x_feasible y y_feasible representan los vértices del polígono que forma la región factible, definida por:

* $𝑥≥0$
* $𝑦≥0$
* $𝑥+𝑦≤20$
* $𝑥≤8$

Estos vértices corresponden a:

* (0, 0)
* (8, 0)
* (8, 12)
* (0, 20)

In [ ]:
plt.figure(figsize=(10,6))

# Coordenadas de los vértices (por intersección de restricciones)
x_feasible = [0, 8, 8, 0]
y_feasible = [0, 0, 12, 20]
plt.fill(x_feasible, y_feasible, color='lightblue', label='Región factible')

# Dibuja las restricciones como líneas
plt.plot(x, y1, label=r'$x + y \leq 20$', color='black') # x+y=20 → frontera de la región factible
plt.axvline(8, label=r'$x \leq 8$', color='green') # x=8 es la restricción del modelo A

# Función objetivo: Z = 5x + 3y
# Se grafican varias líneas con distintos valores de Z (40, 50, 60, 76, 90) para ver cómo "se mueve" la función objetivo.
# Cuando una de estas líneas toca el último punto dentro de la región factible, se alcanza el máximo.
# En este caso, la línea Z = 76 toca exactamente el vértice (8, 12) de la región factible.
for z in [40, 50, 60, 76, 90]:
    y_obj = (z - 5 * x) / 3 # Cuando Z = 76 → 76 = 5x + 3y → y = (76 - 5x) / 3
    plt.plot(x, y_obj, '--', label=fr'$Z={z}$')

# Apariencia del gráfico
plt.xlim(0, 20)
plt.ylim(0, 25)
plt.xlabel('x (Horas Modelo A)')
plt.ylabel('y (Horas Modelo B)')
plt.title('Representación gráfica del problema de optimización')
plt.grid(True)
plt.legend(loc='upper right')
plt.axhline(0, color='gray', linewidth=0.5)
plt.axvline(0, color='gray', linewidth=0.5)

plt.show()